In [1]:
import numpy as np
from scipy.special import owens_t
import matplotlib.pyplot as plt
# get a logger
import logging
logger = logging.getLogger(__name__)
from ERM import fair_adversarial_error_erm
from state_evolution import OverlapSet, fair_adversarial_error_overlaps
%load_ext autoreload
%autoreload 2
from scipy.integrate import quad, dblquad

In [14]:
from data_model import KFeaturesModel
d = 1000
idx_0 = 0
idx_2 = 700
tau = 0
feature_ratios = np.array([0.5,0.5])
features_x = np.array([5,1])
features_theta = np.array([1,1])
sigma_w_content = np.array([1,1])
sigma_delta_content = np.array([1,1])
sigma_upsilon_content = np.array([1,1])
data_model = KFeaturesModel(d, logger, delete_existing=True, normalize_matrices=True,source_pickle_path="",feature_ratios=feature_ratios,features_x=features_x,features_theta=features_theta,Sigma_w_content = sigma_w_content,Sigma_delta_content = sigma_delta_content, Sigma_upsilon_content = sigma_upsilon_content)
# data_model.Sigma_x = data_model.Sigma_x / np.trace(data_model.Sigma_x) * d
rho = data_model.rho



In [15]:
def compute_teacher_margin(y, theta, x):
    z = x.dot(theta)
    return np.mean(y * z / np.sqrt(d))

In [16]:
avg_margin = []

for n in range(10):

    data_set = data_model.generate_data(10000,tau=tau)
    margin = compute_teacher_margin(data_set.y, data_set.theta, data_set.X)

    avg_margin.append(margin)


print(np.mean(avg_margin), np.std(avg_margin))


0.8021510358663887 0.025515779536500416


In [17]:
np.sqrt(2/np.pi) * rho / np.sqrt(rho + tau**2)

0.7978845608028654

In [30]:
from scipy.optimize import minimize

def single_margin(delta, theta, y):
    z = delta.dot(theta)
    return y * z

def constraint(delta, eps):
    return - (np.linalg.norm(delta) - eps)

epsilons = np.array([0,0.1,0.3])
ys = np.array([-1,1])

for n in range(10):

    data_set = data_model.generate_data(10000,tau=tau)
    print("n", n)
    for y in ys:
        for eps in epsilons:

            # minimize single_margin(delta, theta, y) subject to constraint(delta, eps)
            y = 1
            theta = data_set.theta
            eps = 0.1
            delta = np.random.normal(size=d)
            res = minimize(lambda delta: single_margin(delta, theta, y), delta, constraints={'type': 'ineq', 'fun': lambda delta: constraint(delta, eps)})
            delta = res.x

            closed_delta = -eps * theta / np.linalg.norm(theta)
            assert np.allclose(delta, closed_delta, atol=1e-4)

print("Success")

Success


[ 3.20213047e-04 -4.72694143e-03  8.69387912e-05  1.93718280e-03
  2.54613151e-03 -5.50178259e-03  3.20472001e-03  3.47050913e-04
  4.40481118e-03 -5.75408255e-05  5.65319642e-04 -2.43046699e-03
 -1.32559464e-03 -5.83307318e-03 -3.17774768e-03 -1.83399924e-03
 -2.75008213e-03 -4.01639523e-03  4.58655939e-04  3.95908231e-03
  4.81452168e-03  8.71237098e-03  1.12435722e-03 -2.26596321e-03
 -4.59154848e-03  2.28998302e-03  4.89015228e-03 -5.36325212e-03
 -4.33243402e-03 -4.15460740e-04  1.66697598e-03 -1.05933921e-03
  1.49340517e-04 -5.47116110e-04 -5.97758285e-03  5.53773691e-03
 -1.00946937e-03  2.66817117e-03  5.75693430e-03 -2.21970261e-03
  4.51799947e-03 -2.76696756e-03  1.36381525e-03  5.88800839e-04
  1.51886069e-03  1.78709085e-03 -5.58133341e-04  4.49244684e-03
 -3.46828901e-03  3.76805460e-03  2.26202343e-03 -2.50799042e-03
  1.95655914e-03 -4.56581445e-03  7.16089556e-04  5.14030184e-03
  3.85216104e-03 -6.10626057e-03  7.12612056e-04 -2.20694838e-03
  3.28288518e-04 -2.60801

In [27]:
# assert all elements of delta are close to closed_delta
# print(np.linalg.norm(delta - closed_delta))
assert np.allclose(delta, closed_delta, atol=1e-6)

True

In [375]:
data_model.Sigma_theta

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [376]:
data_model.Sigma_x

array([[1.66666667, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.66666667, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.66666667, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.33333333, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.33333333,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.33333333]])

In [377]:
data_set.X

array([[-0.03140859, -0.59517699,  1.10811619, ..., -0.58273646,
        -0.26937393, -0.54461768],
       [ 0.85943968,  0.39133583,  0.21340422, ...,  0.09694138,
        -0.71540863,  0.42064807],
       [ 0.75168628,  2.05204309, -1.00165427, ..., -0.4134435 ,
        -0.4205976 , -0.16374371],
       ...,
       [-0.5278573 , -0.18548004, -2.06440779, ..., -0.77563134,
        -0.49243654, -0.79743468],
       [ 0.79718241, -1.76203604,  2.50416583, ...,  0.31277689,
         1.49971894, -0.94249246],
       [ 0.95488283,  2.21476802, -0.94796962, ..., -0.31363436,
         0.03944832, -0.91545142]])

In [378]:
data_set.theta

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [379]:
# let's compute the average value of the first data-poitn * y and the last data-point * y
# let's extract all the first data_point from data_set.X
first_data_points = data_set.X[:,idx_0]
second_data_points = data_set.X[:,idx_2]

first_mean = np.mean( first_data_points * data_set.y )
second_mean = np.mean( second_data_points * data_set.y )

print(first_mean)
print(second_mean)

0.046814238868614445
0.007676663553179585


In [380]:

def get_usefulness(idx):
    return np.sqrt(2)* (data_set.theta.dot(data_model.Sigma_x))[idx]/( np.sqrt(np.pi *d )* np.sqrt(data_model.rho + tau**2) )

print(get_usefulness(idx_0))
print(get_usefulness(idx_2))


0.042052208700336005
0.0084104417400672


In [381]:
# def integrand(a,b, idx):
#     v = np.array([a,b])
#     va = v[0]
#     vb = v[1]
#     val = vb
#     if idx == 0:
#         val = va
#     return val*np.sign(data_set.theta.dot(v)/np.sqrt(d)) * 1/(2*np.pi*np.sqrt( np.trace(data_model.Sigma_x) ) ) * np.exp(-0.5 * v.dot(np.linalg.inv(data_model.Sigma_x)@v))

# I0 = dblquad(lambda x,y: integrand(x,y,0), -np.inf, np.inf, lambda x: -np.inf, lambda x: np.inf)
# I1 = dblquad(lambda x,y: integrand(x,y,1), -np.inf, np.inf, lambda x: -np.inf, lambda x: np.inf)


In [373]:
# print(I0)
# print(I1)